### Data mining

We access the data given by the professor and store them in a compact dataframe.

In [3]:
%run scripts/data_collection_setup.py

directory = 'data/raw'
df = pd.DataFrame()
for filename in os.listdir(directory):

    dataset = pd.read_csv(directory + '/' + filename)
    dataset = dataset[["Datetime", "Tweet Id", "original_text", "geo"]].dropna(subset=["geo"])
    dataset["geo"] = dataset["geo"].apply(lambda x: x[x.find('place_id')+12:x.find('place_id')+12+16])

    df = pd.concat([df, dataset], ignore_index=True)

We request geolocalization data to the Twitter API using "place_id" and store the results.

In [ ]:
access_token = get_access_token()

update_gs = pd.Series()
for x, y in zip(range(0, len(df["geo"]), 50), range(50, len(df["geo"]), 50)):
    geo_series = df["geo"][x:y].apply(get_geo_data, args=(access_token,))
    update_gs = pd.concat([update_gs, geo_series])
    time.sleep(60*17)

update_gs.to_csv("data/update_gs.csv", index=False)

We build the dataset with only Nigerian tweets.

In [2]:
y = pd.read_csv("data/update_gs.csv").squeeze()

y = y.apply(ast.literal_eval)
y = y.apply(pd.Series)
y = y[['id', 'name', 'place_type', 'full_name', 'country', 'bounding_box', 'centroid']]

df.rename(columns = {'geo' : 'id'}, inplace=True)
tweet_df = df.iloc[:3550].merge(y, right_index=True, left_index=True, validate="1:1")
tweet_df = tweet_df[tweet_df.country=='Nigeria'].sort_values(by="Datetime")
tweet_df.to_csv('data/tweet_df.csv')

We request rainfall data from weather API.

In [ ]:
from itertools import product

regions = [
    'calabar',
    'warri',
    'benin', 
    'lagos',
    'enugu',
    'ikeja',
    'ondo',
    'ibadan',
    'oshogbo',  
    'lokoja',
    'bida',
    'yola', 
    'minna',
    'jos',
    'bauchi',
    'kaduna',
    'yelwa',
    'zaria',
    'maiduguri',
    'kano',
    'gusau',
    'nguru',
    'sokoto',
    'katsina'
]

months = ['2022-{:02d}-08'.format(x) for x in range(1, 12)]
rain_df = pd.Series(tuple(product(months, regions)), index=tuple(product(months, regions))).unstack()
rain_df = rain_df.applymap(rain_request)
rain_df.to_csv('data/rain_df_4.csv')

We get timeseries for daily average precipitations.

In [ ]:
rain_df = pd.read_csv('data/rain_df_2.csv', index_col=0)
rain_df = rain_df.applymap(ast.literal_eval)
rain_df.index = pd.to_datetime(rain_df.index)
rain_df = rain_df.resample('D').fillna('ffill')

for region in rain_df.columns:
    for date in rain_df.index:
        rain_df.loc[date, region] = dict_selection(rain_df.loc[date, region], date)

rain_df = rain_df.dropna()
rain_df.to_csv('data/rain_data.csv')